<a href="https://colab.research.google.com/github/jeisonxm/Data-Science-Project/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importamos todas las librerias que necesitemos

# Sección nueva

In [1]:
from tkinter import *
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy
import tensorflow
import json
import random
import pickle

In [2]:
pip install tflearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 KB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=c3c73fc1b81f868e22a80f43a468e4f3f300e9dc397c92d3f5d96c74a3b7ee93
  Stored in directory: /root/.cache/pip/wheels/65/9b/15/cb1e6b279c14ed897530d15cfd7da8e3df8a947e593f5cfe59
Successfully built tflearn


In [3]:
import tflearn

Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
stemmer = LancasterStemmer()
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Abrimos los datos del archivo json

In [5]:
with open('contenido.json',encoding='utf-8') as archivo:
  datos = json.load(archivo)

print(datos)

FileNotFoundError: ignored

In [ ]:
palabras = []
tags = []
auxX = []
auxY = []

for contenido in datos['contenido']:
  for patrones in contenido['patrones']:
    auxPalabra = nltk.word_tokenize(patrones) # Nltk nos permite trabajar con lenguaje natural
    palabras.extend(auxPalabra)
    auxX.append(auxPalabra)
    auxY.append(contenido["tag"])

    if contenido["tag"] not in tags:
      tags.append(contenido["tag"])

print(palabras,auxX,auxY,sep='\n')

In [ ]:
palabras = [stemmer.stem(w.lower()) for w in palabras if w!='?']  
palabras = sorted(list(set(palabras)))
tags = sorted(tags)
print(palabras)

Ajustamos las salidas de palabras en 1 y 0 para ayudar el entrenamiento de la ia, en pocas palabras nos ayuda a identificar el tag y ayudar a dar una respuesta dependienod de la palabra y relacion que encuentra

In [6]:
entrenamiento = []
salida = []
salidaVacia = [0 for _ in range(len(tags))]

for x, documento in enumerate(auxY): # Enumerate nos permite obtener una palabra y su indice y se maneja con indice
  cubeta = []
  auxPalabra = [stemmer.stem(w.lower()) for w in documento]
  for w in palabras:
    if w in auxPalabra:
      cubeta.append(1)
    else:
        cubeta.append(0)
  filaSalida = salidaVacia[:]
  filaSalida[tags.index(auxY[x])]=1
  entrenamiento.append(cubeta)
  salida.append(filaSalida)
print(entrenamiento, salida, sep='\n')

NameError: ignored

In [ ]:
# Cambiamos las listas normales por listas de numpy para manejar mejor sus dimensiones 
entrenamiento = numpy.array(entrenamiento) 
salida = numpy.array(salida)

tensorflow.compat.v1.reset_default_graph()
red = tflearn.input_data(shape=[None,len(entrenamiento[0])])
red = tflearn.fully_connected(red,10) # Esto es creando 10 neuronas conectadas
red = tflearn.fully_connected(red,10) # SE vuelve a repetir para seguir conectadas entre todas 10
red = tflearn.fully_connected(red,len(salida[0]),activation='softmax') # Aqui con respecto a las probabilidades sacamos el output
red = tflearn.regression(red)

modelo = tflearn.DNN(red)
modelo.fit(entrenamiento,salida,n_epoch=500, batch_size=10) 
# n_epoch= es ver cuantas veces mira la informacion 
# batch_size es la cantidad de palabras que se usan en patrones
modelo.save("modelo.tflearn")

In [ ]:
salida[0]

In [ ]:
def mainBot():
  while True:
    entrada = input("Tu: ")
    cubeta = [0 for _ in range(len(palabras))]
    entradaProcesada = nltk.word_tokenize(entrada)
    entradaProcesada = [stemmer.stem(palabra.lower()) for palabra in entrada]
    for palabraIndividual in entradaProcesada:
      for i, palabra in enumerate(palabras):
        if palabra == palabraIndividual:
          cubeta[i] = 1
    resultados = modelo.predict([numpy.array(cubeta)])
    resultadosIndices = numpy.argmax(resultados)
    tag = tags[resultadosIndices]
    for tagAux in datos['contenido']:
      if tagAux['tag'] == tag:
        respuesta = tagAux['respuestas']
    print("Bot: ", random.choice(respuesta))
mainBot()

Bot:  te estare esperando
Bot:  te estare esperando
Bot:  te estare esperando
Bot:  cuidate
